<a href="https://colab.research.google.com/github/Dockaun/Dockaun/blob/main/June14test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('lesson_script_variations(1).csv')

# Display the first few rows to verify
print(df.head())

# Check the column names and data types
print(df.dtypes)


                                             context  \
0  Welcome to today's lesson on supervised learni...   
1  Welcome to today's lesson on supervised learni...   
2  Welcome to today's lesson on supervised learni...   
3  Welcome to today's lesson on supervised learni...   
4  Welcome to today's lesson on supervised learni...   

                                            question  \
0  If we have a dataset of various kinds of apple...   
1  If we have a dataset of various kinds of apple...   
2  If we have a dataset of various kinds of apple...   
3  If we have a dataset of various kinds of apple...   
4  If we have a dataset of various kinds of apple...   

                                              answer feedback_type  
0  That's right! One method is to use labeled dat...       correct  
1  By using labeled data, AI can learn to recogni...       correct  
2  AI distinguishes between fruits by learning fr...       correct  
3  Interesting thought. However, the most effecti.

In [2]:
from transformers import BertTokenizerFast

# Initialize the tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Tokenize the data
def tokenize_function(row):
    return tokenizer(row['context'], row['question'], truncation=True, padding='max_length', max_length=512)

# Apply the function to each row in the DataFrame
tokenized_data = df.apply(tokenize_function, axis=1)

# Extract input_ids and attention_mask
input_ids = [item['input_ids'] for item in tokenized_data]
attention_mask = [item['attention_mask'] for item in tokenized_data]

print("Tokenization successful")


Tokenization successful


In [3]:
import torch

# Convert lists to tensors
input_ids = torch.tensor(input_ids)
attention_mask = torch.tensor(attention_mask)

print("Tensors created:")
print("Input IDs shape:", input_ids.shape)
print("Attention Mask shape:", attention_mask.shape)


Tensors created:
Input IDs shape: torch.Size([54, 512])
Attention Mask shape: torch.Size([54, 512])


In [4]:
# Print the column names in the DataFrame
print(df.columns)


Index(['context', 'question', 'answer', 'feedback_type'], dtype='object')


In [5]:
# Convert categorical labels to integers
labels_tensor = torch.tensor(df['feedback_type'].factorize()[0])

print("Labels prepared:", labels_tensor.shape)


Labels prepared: torch.Size([54])


In [6]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Tokenize the data
def tokenize_function(row):
    return tokenizer(row['context'], row['question'], truncation=True, padding='max_length', max_length=512)

# Apply the function to each row in the DataFrame
tokenized_data = df.apply(tokenize_function, axis=1)

# Convert the results into tensors
input_ids = torch.tensor([item['input_ids'] for item in tokenized_data])
attention_mask = torch.tensor([item['attention_mask'] for item in tokenized_data])


In [7]:
labels_tensor = torch.tensor(df['feedback_type'].factorize()[0])


In [8]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)

# Instantiate the dataset
dataset = CustomDataset(input_ids, attention_mask, labels_tensor)


In [9]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(labels_tensor.numpy())))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
!pip install accelerate -U


In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=3,              # Total number of training epochs
    per_device_train_batch_size=8,   # Batch size per device during training
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10
)


In [12]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

# Start training
trainer.train()


Step,Training Loss
10,1.091800
20,1.111200


TrainOutput(global_step=21, training_loss=1.1026318470637004, metrics={'train_runtime': 1057.58, 'train_samples_per_second': 0.153, 'train_steps_per_second': 0.02, 'total_flos': 42624373671936.0, 'train_loss': 1.1026318470637004, 'epoch': 3.0})

In [13]:
from torch.utils.data import random_split

# Assuming 'dataset' is your full dataset and you want an 80-20 train-validation split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


In [14]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)


In [15]:
results = trainer.evaluate()
print(results)


{'eval_loss': 1.1029601097106934, 'eval_runtime': 42.6219, 'eval_samples_per_second': 0.258, 'eval_steps_per_second': 0.047}
